In [19]:
!pip3 install elasticsearch

In [20]:
!pip3 install elasticsearch-dsl

In [21]:
import sys
sys.path.insert(0, '../../../BERT-FAQ/')

# import required libraries
from elasticsearch_dsl import Index, Document, Integer, Text, analyzer, Keyword, Double
from elasticsearch_dsl.connections import connections
from elasticsearch import Elasticsearch, helpers

from evaluation import get_relevance_label_df
from shared.utils import load_from_json
from shared.utils import dump_to_json
from shared.utils import make_dirs
from indexer import ingest_data
from indexer import QA
import logging

In [22]:
# get list of query answer pairs

query_answer_pairs_filepath = '../../../BERT-FAQ/data/MentalFAQ/query_answer_pairs.json'
relevance_label_df = get_relevance_label_df(query_answer_pairs_filepath)
faq_qa_pair_df = relevance_label_df[relevance_label_df['query_type'] == 'faq']
faq_qa_pairs = faq_qa_pair_df.T.to_dict().values()

In [23]:
# check relevance_label_df
relevance_label_df

,label,query_type,question,answer,id
0,1,faq,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...,1
1,1,faq,Who does mental illness affect?,It is estimated that mental illness affects 1 ...,2
2,1,faq,What causes mental illness?,It is estimated that mental illness affects 1 ...,3
3,1,faq,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...,4
4,1,faq,Can people with mental illness recover?,"When healing from mental illness, early identi...",5
...,...,...,...,...,...
191,1,user_query,How do I know if I'm drinking too much?,Sorting out if you are drinking too much can b...,192
192,1,user_query,"If cannabis is dangerous, why are we legalizin...","Cannabis smoke, for example, contains cancer-c...",193
193,1,user_query,How can I convince my kids not to use drugs?,You can't. But you can influence their capacit...,194
194,1,user_query,What is the legal status (and evidence) of CBD...,Cannabidiol or CBD is a naturally occurring co...,195


In [24]:
# check faq_qa_pair_df
faq_qa_pair_df

,label,query_type,question,answer,id
0,1,faq,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...,1
1,1,faq,Who does mental illness affect?,It is estimated that mental illness affects 1 ...,2
2,1,faq,What causes mental illness?,It is estimated that mental illness affects 1 ...,3
3,1,faq,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...,4
4,1,faq,Can people with mental illness recover?,"When healing from mental illness, early identi...",5
5,1,faq,What should I do if I know someone who appears...,Although this website cannot substitute for pr...,6
6,1,faq,How can I find a mental health professional fo...,Feeling comfortable with the professional you ...,7
7,1,faq,What treatment options are available?,Just as there are different types of medicatio...,8
8,1,faq,"If I become involved in treatment, what do I n...",Since beginning treatment is a big step for in...,9
9,1,faq,What is the difference between mental health p...,There are many types of mental health professi...,10


In [25]:
# convert to list
faq_qa_pairs = list(faq_qa_pairs)

In [26]:
# check first 5 entries in faq_qa_pairs
faq_qa_pairs[:5]

[{'label': 1,
  'query_type': 'faq',
  'question': 'What does it mean to have a mental illness?',
  'answer': 'Mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.\nMental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital.\nIt is important to know that mental illnesses are medical conditions that have nothing to do with a personâ€™s character, intelligence, or willpower. Just as diabetes is a disorder of the pancreas, mental illness is a me

In [27]:
try:
    
    es = connections.create_connection(hosts=['localhost'])
    
    # Index data to Elasticsearch 
    index_name = "mentalfaq"
    
    # Initialize index (only perform once)
    index = Index(index_name)

    # Define custom settings
    index.settings(
        number_of_shards=1,
        number_of_replicas=0
    )

    # Delete the index, ignore if it doesn't exist
    index.delete(ignore=404)

    # Create the index in Elasticsearch
    index.create()

    # Register a document with the index
    index.document(QA)

    # Ingest data to Elasticsearch
    ingest_data(faq_qa_pairs, es=es, index=index_name)

    print("Finished indexing {} records to {} index".format(len(faq_qa_pairs), index_name))

except Exception:
    logging.error('exception occured', exc_info=True)


100%|██████████| 98/98 [00:00<00:00, 49021.08it/s]


Finished indexing 98 records to mentalfaq index


c:\Users\Usuario\AppData\Local\Programs\Python\Python39\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)
